<a href="https://colab.research.google.com/github/GuoZhuyin/Zhuyin.Asia/blob/main/DeepLearning/Farmland/cropimgbymodel_TEAM_855_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 訓練完成的模型進行切圖

In [ ]:
import PIL
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
classes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato', 'building', 'mountain', 'sky']
num_classes = len(classes)
print(num_classes)

原有14個作物類別外，另外又加入'build','mountain','sky'，讓模型可以辨識那些為不需要的類別，所以共17個類別

In [ ]:
from torchvision import models
#model = models.inception_v3(pretrained=False)
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
model.aux_logits = False
model.fc =  nn.Linear(model.fc.in_features, num_classes)

透過inception_v3進行切圖

In [ ]:
model = model.cuda()
model = nn.DataParallel(model) #cuda使用

In [ ]:
model.load_state_dict(torch.load('../model/InV3_TEAM_855_5.0.pth')) #InV3_TEAM_855_5.0.pth為訓練完成儲存的模型

輸入的模型參數

In [ ]:
model.eval()

In [ ]:
mydir = "FITV3"

建立資料夾暫時將截圖後的圖片放在該資料夾中

In [ ]:
if not os.path.isdir(mydir):
    os.mkdir(mydir)

In [ ]:
folders=[ 'carrot', 'corn', 'dragonfruit', 'peanut', 'pumpkin', 'soybean' ]
print(len(folders))
#'banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato'

輸入的資料夾位置

In [ ]:
nonfield=['building', 'mountain', 'sky']
print(len(nonfield))

In [ ]:
class_to_index={'banana': 0,
 'bareland': 1,
 'building': 2,
 'carrot': 3,
 'corn': 4,
 'dragonfruit': 5,
 'garlic': 6,
 'guava': 7,
 'mountain': 8,
 'peanut': 9,
 'pineapple': 10,
 'pumpkin': 11,
 'rice': 12,
 'sky': 13,
 'soybean': 14,
 'sugarcane': 15,
 'tomato': 16}

In [ ]:
index_to_class = {v: k for k, v in class_to_index.items()}

In [ ]:
import os
for sub in nonfield:
    subpath=os.path.join(mydir, sub)
    if not os.path.isdir(subpath):
      os.mkdir(subpath)

如果被預測為該類別則放入FITV3/subpath

In [ ]:
for sub in folders:
    subpath=os.path.join(mydir, sub+"_out")
    if not os.path.isdir(subpath):
      os.mkdir(subpath)
    subpath=os.path.join(mydir, sub)
    if not os.path.isdir(subpath):
      os.mkdir(subpath)

如果被預測出來不是該類別且不是'build','mountain','sky'則放入FITV3/subpath_out

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
import glob
from pathlib import Path
from PIL import Image 
for f in folders:
    landtype_index=class_to_index[f]
    subpath=os.path.join(f, "*")
    files = glob.glob(subpath)
    print(f"Categroy:{f}")
    for fname in files:
      print(fname)
      img_name = Path(fname).stem
      ext_name = Path(fname).suffix
      print(img_name, ext_name, sep=" ")

In [ ]:
from pathlib import Path
def scan_image(fname, landtype_index):
    count = 0
    img_name = Path(fname).stem
    ext_name = Path(fname).suffix
    im = Image.open(fname)
    width, height = im.size   # Get dimensions
    new_width = 512
    new_height = 512
    stride = 512
    for startx in range(width//8, width*7//8-new_width, stride):
      for starty in range(height//8, height*7//8-new_height, stride):
          left = startx
          top = starty
          right = startx + new_width
          bottom = starty + new_height
          subname = f"{img_name}-{count:03}{ext_name}"
          count +=1
          im2 = im.crop((left, top, right, bottom))
          input_tensor = test_transform(im2)
          input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
          input_batch = input_batch.to('cuda')
          output = model(input_batch)
          probabilities = torch.nn.functional.softmax(output[0], dim=0).tolist()
          det = argmax(probabilities)
          if det ==2 or det ==8 or det ==13:#building, mountain sky
             subfolder = index_to_class[det]
          elif probabilities[landtype_index] > 0.5: #數值可能性必須大於0.5才進行分類
             subfolder = index_to_class[landtype_index]
          else:
             subfolder = index_to_class[landtype_index]+'_out' 
          name2 = os.path.join(mydir, subfolder, subname)
          print(name2)
          im2.save(name2)
          break

In [ ]:
def argmax(lst): return max(range(len(lst)), key=lst.__getitem__)

In [ ]:
a=[ 4, 5, 6, 8, 2]

In [ ]:
argmax(a)

In [ ]:
import glob
from PIL import Image 
for f in folders:
    landtype_index=class_to_index[f]
    subpath=os.path.join(f, "*")
    files = glob.glob(subpath)
    print(f"Categroy:{f}")
    for fname in files:
      print(fname)
      scan_image(fname, landtype_index)